In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 50      # number of variables
num_ineq = 50     # number of constraints
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# generate parameters
Q = torch.from_numpy(np.diag(np.random.random(size=num_var))).float().cuda()
p = torch.from_numpy(np.random.random(num_var)).float().cuda()
A = torch.from_numpy(np.random.normal(scale=1, size=(num_ineq, num_var))).float().cuda()

In [5]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 256
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [7]:
from src.problem import msConvexQuadratic
model = msConvexQuadratic(Q.cpu().numpy(), p.cpu().numpy(), A.cpu().numpy())

## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_50-50.csv")

 56%|████████████████████████████████████████████▍                                  | 563/1000 [01:53<01:30,  4.83it/s]

ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_dimen' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object
ERROR: Unable to clone Pyomo component attribute. Component 'OrderedScalarSet'
contains an uncopyable field '_init_values' (<class
'pyomo.core.base.set.TuplizeValuesInitializer'>).  Setting field to `None` on
new object


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:24<00:00,  4.88it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -6.598289         27.415983      0.141020
std       0.679915          4.354498      0.011523
min      -8.946865         12.853340      0.122267
25%      -7.103386         24.368541      0.132108
50%      -6.618789         27.254992      0.139096
75%      -6.098403         30.461335      0.147027
max      -4.113986         39.903281      0.213240
Number of infeasible solution: 1000


## Heuristic - N1

In [8]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n1_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [33:55<00:00,  2.04s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   6.581430e+19               0.0      2.027064
std    4.115802e+19               0.0      0.758049
min    3.538812e+01               0.0      0.132121
25%    4.181990e+19               0.0      1.496810
50%    5.541559e+19               0.0      1.938578
75%    7.653952e+19               0.0      2.484729
max    5.069081e+20               0.0      5.301784
Number of infeasible solution: 0


## Learnable Rounding

In [10]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [12]:
# set problem
import neuromancer as nm
from src.problem import nmConvexQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmConvexQuadratic(["b", "x_rnd"], Q, p, A, penalty_weight)

In [13]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 8105.16
Epoch 1, Validation Loss: 2460.32
Epoch 2, Validation Loss: 344.46
Epoch 3, Validation Loss: 229.78
Epoch 4, Validation Loss: 180.33
Epoch 5, Validation Loss: 184.75
Epoch 6, Validation Loss: 162.84
Epoch 7, Validation Loss: 162.88
Epoch 8, Validation Loss: 140.48
Epoch 9, Validation Loss: 136.81
Epoch 10, Validation Loss: 153.91
Epoch 11, Validation Loss: 154.13
Epoch 12, Validation Loss: 133.18
Epoch 13, Validation Loss: 135.87
Epoch 14, Validation Loss: 128.70
Epoch 15, Validation Loss: 114.85
Epoch 16, Validation Loss: 107.65
Epoch 17, Validation Loss: 119.52
Epoch 18, Validation Loss: 97.29
Epoch 19, Validation Loss: 101.99
Epoch 20, Validation Loss: 113.70
Epoch 21, Validation Loss: 110.49
Epoch 22, Validation Loss: 108.35
Epoch 23, Validation Loss: 100.44
Epoch 24, Validation Loss: 102.84
Epoch 25, Validation Loss: 105.04
Epoch 26, Validation Loss: 97.01
Epoch 27, Validation Loss: 102.15
Epoch 28, Validation Loss: 101.61
Epoch 29, Validation Los

In [14]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 76.64it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     30.028164          0.012865      0.002660
std       4.428941          0.113380      0.000705
min      26.986815          0.000000      0.000998
25%      28.973149          0.000000      0.002002
50%      28.973149          0.000000      0.002512
75%      29.206850          0.000000      0.003002
max      98.188925          1.716585      0.009602
Number of infeasible solution: 23


## Learnable Threshold

In [15]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [16]:
# hyperparameters
penalty_weight = 50  # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [18]:
# set problem
import neuromancer as nm
from src.problem import nmConvexQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmConvexQuadratic(["b", "x_rnd"], Q, p, A, penalty_weight)

In [19]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 620.86
Epoch 1, Validation Loss: 884.61
Epoch 2, Validation Loss: 82.56
Epoch 3, Validation Loss: 71.43
Epoch 4, Validation Loss: 68.33
Epoch 5, Validation Loss: 49.42
Epoch 6, Validation Loss: 46.40
Epoch 7, Validation Loss: 54.44
Epoch 8, Validation Loss: 50.86
Epoch 9, Validation Loss: 44.80
Epoch 10, Validation Loss: 47.08
Epoch 11, Validation Loss: 37.76
Epoch 12, Validation Loss: 41.17
Epoch 13, Validation Loss: 60.02
Epoch 14, Validation Loss: 52.21
Epoch 15, Validation Loss: 65.79
Epoch 16, Validation Loss: 50.45
Epoch 17, Validation Loss: 42.37
Epoch 18, Validation Loss: 40.77
Epoch 19, Validation Loss: 50.37
Epoch 20, Validation Loss: 56.23
Epoch 21, Validation Loss: 36.67
Epoch 22, Validation Loss: 33.13
Epoch 23, Validation Loss: 57.98
Epoch 24, Validation Loss: 38.94
Epoch 25, Validation Loss: 38.92
Epoch 26, Validation Loss: 48.42
Epoch 27, Validation Loss: 36.59
Epoch 28, Validation Loss: 36.28
Epoch 29, Validation Loss: 40.72
Epoch 30, Validati

In [20]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 64.31it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     30.658783          0.053647      0.003111
std       3.630421          0.197854      0.000928
min      16.456278          0.000000      0.001001
25%      28.798093          0.000000      0.002506
50%      30.788295          0.000000      0.003000
75%      30.990768          0.000000      0.003531
max      47.348710          2.143152      0.006536
Number of infeasible solution: 124
